In [ ]:
import datetime as dt
import numpy as np
from scipy.stats import poisson
import matplotlib.pyplot as plt
from define.randchans import RandomPauliChannel
from define.QECCLfid import uncorrectable as uc
from define import qcode as qc
from define import fnames as fn
from define import globalvars as gv
import matplotlib
matplotlib.use("Agg")
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
qcode = qc.QuantumErrorCorrectingCode("Steane")
qc.Load(qcode)

In [ ]:
mu = np.mean(qcode.weightdist[qcode.PauliCorrectableIndices])/2
np.random.normal(mu,0.5)

In [ ]:
# plt.plot(np.arange(8),poisson.pmf(np.arange(8), 0),0.1**np.arange(10),marker="o",linestyle="None")

In [ ]:
# probs = poisson.pmf(qcode.weightdist, np.abs(np.random.normal(mu,mu/3)))
# plt.plot(np.sort(qcode.weightdist), probs[np.argsort(qcode.weightdist)],marker="o",linestyle="None")
# plt.plot(np.sort(qcode.weightdist), (1/7)**np.sort(qcode.weightdist),marker="x",linestyle="None")
# plt.yscale('log')

In [ ]:
errdist = RandomPauliChannel({"method":1, "mu":mu, "weightdist":qcode.weightdist})

In [ ]:
infid = 0.2
errdist[0] = 1 - infid
errdist[1:] = infid * errdist[1:]/np.sum(errdist[1:])

In [ ]:
errdist

In [ ]:
PauliDistributionPlot(errdist, qcode, nreps=5, max_weight=3)

In [ ]:
for w in range(np.max(qcode.weightdist)+1):
    print("Total probability of weight {} errors = {}:\n{}".format(w, np.sum(errdist[qcode.weightdist==w]), errdist[qcode.weightdist==w]))

In [1]:
import numpy as np
from define import qcode as qc
from define.QECCLfid import uncorrectable as uc
from analyze.plots import PauliDistributionPlot

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/matplotlib-3.0.2-py3.7-macosx-10.9-x86_64.egg/mpl_toolkits/axes_grid/__init__.py:12: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  obj_type='module')


In [2]:
qcode = qc.QuantumErrorCorrectingCode("Steane")
qc.Load(qcode)
qc.PrepareSyndromeLookUp(qcode)

In [3]:
q1 = 0.8; q2 = 0.2; infid = 0.1; n = qcode.N
single_qubit_errors = np.array([1 - infid, infid/3, infid/3, infid/3], dtype=np.double)

In [4]:
iid_error_dist = uc.GetErrorProbabilities(qcode.PauliOperatorsLST, single_qubit_errors, 0)
full_process_infid = 1 - iid_error_dist[0]
print("Sum of IID error probabilities = {}, Infidelity = {}.".format(np.sum(iid_error_dist), full_process_infid))

Sum of IID error probabilities = 1.0, Infidelity = 0.5217030999999999.


In [5]:
n_two_qubit_errors = np.int(0.1 * qcode.group_by_weight[2].size)
two_qubit_errors = np.random.choice(qcode.group_by_weight[2], size=n_two_qubit_errors)

In [6]:
corr_error_dist = np.zeros(iid_error_dist.size, dtype=np.double)
corr_error_dist[two_qubit_errors] = np.abs(np.random.normal(0.1 * 4**n * full_process_infid, 0.1 * 4**n * full_process_infid, size=(n_two_qubit_errors,)))
corr_error_dist[0] = 1 - full_process_infid
corr_error_dist[two_qubit_errors] = full_process_infid * corr_error_dist[two_qubit_errors]/np.sum(corr_error_dist[two_qubit_errors])
corr_error_dist = corr_error_dist/np.sum(corr_error_dist)

In [7]:
print("Sum of CORR error probabilities = {}, Infidelity = {}".format(np.sum(corr_error_dist), 1-corr_error_dist[0]))

Sum of CORR error probabilities = 1.0000000000000002, Infidelity = 0.5060473905145697


In [8]:
pauli_error_dist = q1 * iid_error_dist + q2 * corr_error_dist

In [10]:
PauliDistributionPlot(qcode, pauli_error_dist, nreps=5, max_weight=3,outdir="./../../temp/", channel="linear_sum")

Leading by weight:
{0: array([0]), 1: array([4120,    8, 1025, 1538, 1546]), 2: array([ 9428,  9565,  8402,   432, 13515]), 3: array([   1, 9596, 9611, 9613, 9635])}
Operator labels:
{0: ['IIIIIII'], 1: ['IIIIIIZ', 'IIIIIZI', 'IIIIIIX', 'IIIIIXI', 'IIIIIYI'], 2: ['XIIIYII', 'IIYZIII', 'IIIIYIX', 'IZIIIIZ', 'IIIIYIZ'], 3: ['XIXIXII', 'XIZIXII', 'IIZZYII', 'IIYZZII', 'IIZIYZI']}
pauliprobs[leading_by_weight[0]] = [0.48142804]
pauliprobs[leading_by_weight[1]] = [0.01417176 0.01417176 0.01417176 0.01417176 0.01417176]
pauliprobs[leading_by_weight[2]] = [0.01446999 0.01231397 0.00892978 0.00811274 0.00781329]
